In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [19]:
#Step 1: Load libraries for the U-net Model


import cv2
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow as tf

tf.keras.layers.Conv1D(
    filters,
    kernel_size,
    strides=1,
    padding='valid',
    data_format='channels_last',
    dilation_rate=1,
    groups=1,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)

In [2]:
def conv_block(x, filter_size, size, stride, batch_norm=False):
    
    conv = layers.Conv1D(size, filter_size, strides = stride, padding="same", kernel_initializer = 'he_normal')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization()(conv)
    conv = layers.Activation("relu")(conv)
    
    return conv

In [20]:
def oleg_model(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    
    block_1 = conv_block(inputs1, 1, 64, 1, batch_norm=True)
    block_2 = conv_block(block_1, 3, 64, 1, batch_norm=True)
    block_3 = conv_block(block_2, 1, 128, 1, batch_norm=True)
    block_4 = conv_block(block_3, 3, 128, 1, batch_norm=True)
    block_5 = conv_block(block_4, 1, 256, 1, batch_norm=True)
    block_6 = conv_block(block_5, 3, 256, 1, batch_norm=True)
    block_7 = conv_block(block_6, 1, 512, 1, batch_norm=True)
    block_8 = conv_block(block_7, 3, 512, 1, batch_norm=True)
    
    average_pooling = AveragePooling1D(pool_size=2, strides=2, padding='same')(block_8)
    flatten = tf.reshape(average_pooling,[-1])
    oleg_final = layers.Activation("linear")(flatten)
    
   
#     conv_final = layers.Conv1D(NUM_CLASSES, kernel_size=(conv)
#     conv_final = layers.BatchNormalization(axis=3)(conv_final)
#     conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel
    
    model = models.Model(inputs1, oleg_final, name="TargetModel")
    return model

In [21]:
model = oleg_model(input_shape = (2,1500))

In [22]:
model.summary()

Model: "TargetModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 2, 1500)]         0         
                                                                 
 conv1d_49 (Conv1D)          (None, 2, 64)             96064     
                                                                 
 batch_normalization_48 (Bat  (None, 2, 64)            256       
 chNormalization)                                                
                                                                 
 activation_49 (Activation)  (None, 2, 64)             0         
                                                                 
 conv1d_50 (Conv1D)          (None, 2, 64)             12352     
                                                                 
 batch_normalization_49 (Bat  (None, 2, 64)            256       
 chNormalization)                                      